# Topic Modeling (토픽 모델링)

토픽(topic)은 한국어로는 주제라고 합니다. 토픽 모델링(Topic Modeling)이란 기계 학습 및 자연어 처리 분야에서 토픽이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델중 하나로, 텍스트 본문의 숨겨진 의미 구조를 발견하기 위해 사용되는 텍스트 마이닝 기법입니다. 

## 1) 잠재 의미 분석 (Latent Semantic Analysis, LSA)

LSA 는 정확히는 토픽 모델링을 위해 최적화 된 알고리즘은 아니다, 토픽 모델링이라는 분야에 아이디어를 제공한 알고리즘 정도로 볼 수 있다. 

BoW에 기반판 DTM 이나 TF-IDF는 기본적으로 단어의 빈도 수를 이용한 수치화 방법이기 때문에 단어의 의미를 고려 하지 못한다는 단점이 있다. (이를 토픽 모델링 관점에서는 단어의 토픽을 고려하지 못한다고 한다.) 이를 위한 대안으로 DTM의 잠대된 (Latent) 의미를 이끌어내는 방법으로 잠재 의미 분석 (Latent Semantic Analysis, LSA)라는 방법이 있다. 잠재 의미 분석 (Latent Semantic Indexing) 이라 하기도 한다.   

이 방법을 이해하기 위해서는 선대의 특이값 분해 (Singular Value Decomposition, SVD)를 이해해야 한다. 

### 1. 특이값 분해 (Singular Value Decomposition, SVD)

시작하기 앞서, 여기서의 특이값 분해(SVD)는 실수 벡터 공간에 한정하여 내용을 설명함을 명시한다. SVD란 A가 m x n 행렬일때, 다음과 같이 3개의 행렬의 곱으로 분해 (decomposition)하는 것을 말합니다.  

A = U*Sigma(V)
여기서 각 3개의 행렬은 다음과 같은 조건을 만족합니다.
U:m×m 직교행렬 (AAT=U(ΣΣT)UT)
V:n×n 직교행렬 (ATA=V(ΣTΣ)VT)
Σ:m×n 직사각 대각행렬

여기서 직교행렬 (orthogonal matrix)란 자신과 자신의 전치 행렬(transposed matrix)의 곱 또는 이를 반대로 곱한 결과가 단위행렬 (identify matrix)이 되는 행렬을 말한다. 또한 대각행렬(digonal matrix)란 주 대각선을 제외한 곳의 원소가 모두 0인 행렬을 의미한다.  

이때 SVD로 나온 대각 행렬의 대각 원소의 값을 행렬 A의 특이값(singular value)라고 한다.

#### 1-1) 전치행렬 (Transposed Matrix)

전치 행렬(Transposed matrix)는 원래의 행렬에서 행과 열을 바꾼 행렬이다. 즉, 주 대각선을 축으로 반사 대칭을 하여 얻는 행렬이라 할 수 있다. 기호는 기존의 행렬 표현의 우측위에 T를 붙여 표기한다. 예를 들어 기존의 행렬을 M이라 하면 전치 행렬을 M^T 와 같이 표현한다.  

In [7]:
import numpy as np

M = np.array([1,2,3,4,5,6]).reshape(3,2)
print(M)
print(M.T)

[[1 2]
 [3 4]
 [5 6]]
[[1 3 5]
 [2 4 6]]


numpy 에 궁금하다면 제 깃헙 레파지도리의 Python/Numpy/NumpyAll.ipynb 에서 참고하실수 있습니다.

#### 1-2) 단위행렬 (Identity Matrix)

단위 행렬(identity matrix)는 주 대각선의 우너소가 모두 1이며 나머지 원소는 모두 0인 정사각 행려을 말한다. 보통 줄여 대문자 I로 표기한다. 2*2 단위행렬과 3*3 단위행렬을 아래와 같은 형태를 하고 있다.

In [10]:
I2 = np.eye(2)
I3 = np.eye(3)

print(I2)
print(I3)
print(np.dot(M, I2))
print(np.dot(M.T, I3))

[[1. 0.]
 [0. 1.]]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[1. 2.]
 [3. 4.]
 [5. 6.]]
[[1. 3. 5.]
 [2. 4. 6.]]


#### 1-3) 역행렬(Inverse Matrix)
단위 행렬(identity matrix)를 이해 했다면 역행렬을 정의할 수 있다.  
만약 행렬 A와 어떤 행렬을 곱할 때 결과로서 단위행렬이 나온다면 이때의 어떤 행렬을 A의 역행렬이라 하며 A^-1 라고 표현한다.

In [16]:
x = np.array([2,4,5,2]).reshape(2,2)
# x의 역행렬은??
print(x)
x_inverse = np.linalg.inv(x)
print(x_inverse)
print(np.dot(x, x_inverse))

[[2 4]
 [5 2]]
[[-0.125   0.25  ]
 [ 0.3125 -0.125 ]]
[[1. 0.]
 [0. 1.]]


#### 1-4) 직교 행렬(Orthogonal matrix)
다시 직교 행렬(orthogonal matrix)의 정의로 돌아가서, 실수 n x n행렬 A에 대해서 A x A.T = I 를 만족하면서 A.T x A = I 또한 만족하는 행렬 A를 직교 행렬이라고 한다. 그런데 역행렬의 정의를 다시 생각해보면, 결국 직교 행렬을 A^-1 = A.T 를 만족한다.  

#### 1-5) 대각행렬(Diagonal matrix)
대각행렬(diagonal matrix)는 주 대각선을 제외한 곳의 원소가 모두 0인 행렬을 말한다. 아래의 그림에서는 주 대각선의 원소를 a라 표현하고 있다. 만약 대각 행렬 Σ가 3 × 3 행렬이라면, 다음과 같은 모양을 가진다.  

In [17]:
x = np.arange(9).reshape((3,3))
print(x)

[[0 1 2]
 [3 4 5]
 [6 7 8]]


In [18]:
np.diag(x)

array([0, 4, 8])

In [19]:
np.diag(x, k=1)

array([1, 5])

In [20]:
np.diag(x, k=-1)

array([3, 7])

In [21]:
np.diag(np.diag(x))

array([[0, 0, 0],
       [0, 4, 0],
       [0, 0, 8]])

##### Parameters:
v : array_like
If v is a 2-D array, return a copy of its k-th diagonal. If v is a 1-D array, return a 2-D array with v on the k-th diagonal.

k : int, optional
Diagonal in question. The default is 0. Use k>0 for diagonals above the main diagonal, and k<0 for diagonals below the main diagonal.

In [22]:
y = np.arange(12).reshape((4,3))

In [23]:
print(y)

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]


In [26]:
print(np.diag(np.diag(y))) 
##? 아래 000은 왜 안깔리지,,

[[0 0 0]
 [0 4 0]
 [0 0 8]]


### 2. 절단된 SVD (Truncated SVD)
위에서 설명한 SVD를 풀 SVD(full SVD)라고 합니다. 하지만 LSA의 경우 풀 SVD에서 나온 3개의 행렬에서 일부 벡터들을 삭제시킨 절단된 SVD(truncated SVD)를 사용하게 됩니다. 그림을 통해 이해해보도록 하겠습니다.  

![](https://wikidocs.net/images/page/24949/svd%EC%99%80truncatedsvd.PNG)

절단된 SVD는 대각 행렬  Σ의 대각 원소의 값 중에서 상위값 t개만 남게 됩니다. 절단된 SVD를 수행하면 값의 손실이 일어나므로 기존의 행렬 A를 복구할 수 없습니다. 또한, U행렬과 V행렬의 t열까지만 남깁니다. 여기서 t는 우리가 찾고자하는 토픽의 수를 반영한 하이퍼파라미터값입니다. 하이퍼파라미터란 사용자가 직접 값을 선택하며 성능에 영향을 주는 매개변수를 말합니다. t를 선택하는 것은 쉽지 않은 일입니다. t를 크게 잡으면 기존의 행렬 A로부터 다양한 의미를 가져갈 수 있지만, t를 작게 잡아야만 노이즈를 제거할 수 있기 때문입니다.  

이렇게 일부 벡터들을 삭제하는 것을 데이터의 차원을 줄인다고도 말하는데, 데이터의 차원을 줄이게되면 당연히 풀 SVD를 하였을 때보다 직관적으로 계싼 비용이 낮아지는 효과를 얻을수 있다   

하지만 계산비용이 낮아지는 것 외에도 상대적으로 중요하지 않은 정보를 삭제하는 효과를 발생하기 때문에, 이는 영상처리 분야에서는 노이즈를 제거한다는 의미를 갖고 자연어 처리분야에서는 설명력이 낮은 정보를 삭제하고 설명력이 높은 정보를 남긴다는 의미를 갖는다. 즉, 다시말하면 기존 행렬에서는 드러나지 않았던 심층적인 의미를 확인 가능케 한다는것이다.  

### 3. 잠재 의미 분석 (Latent Semantic Analysis, LSA)

기존의 DTM이나 DTM에 단어의 중요도에 따라 가중치를 주었던 TF-IDF행렬은 단어의 의미를 전혀 고려하지 않는다는 단점을 갖고 있다. LSA는 기본적으로 DTM이나 TF-IDF행렬에 절단된 SVD(truncated SVD)를 사용하여 차원을 축소하고 단어들의 잠재적인 의미를 끌어낸다.  

실습을 통해 이해보도록 하자.  

![gg](datasets/img/20191017_151720.png)

In [27]:
import numpy as np
A=np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
np.shape(A)

(4, 9)

이를 통해 4x9 크기의 DTM이 생성되었다. 이에 대하여 Full SVD를 수행해보자, 단 여기서는 대각행렬의 변수명을 시그마기호가 아닌 S를 사용한다. 또한 V의 전치 행렬을 VT라 한다.

In [28]:
U, s, VT = np.linalg.svd(A, full_matrices = True)

In [30]:
print(U)

[[-2.39751712e-01  7.51083898e-01  5.48519247e-17 -6.15135834e-01]
 [-5.06077194e-01  4.44029376e-01 -8.22778870e-17  7.39407727e-01]
 [-8.28495619e-01 -4.88580485e-01 -7.47642703e-17 -2.73649629e-01]
 [-9.04299898e-17 -4.11929620e-17  1.00000000e+00  7.41190750e-17]]


In [31]:
print(U.round(2))
np.shape(U)

[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]


(4, 4)

4x4 크기를 갖는 직교행렬 U가 생성되었다. 이제 대각 행렬 S를 확인해보자.

In [32]:
print(s.round(2))
print(np.shape(s))

[2.69 2.05 1.73 0.77]
(4,)


Numpy 의 linalg.svd()는 특이값 분해의 결과로 대각 행렬이 아닌 특이값의 리스틀 반환한다. 그러므로 앞서 본 수식의 형식으로 보려면 이를 다시 대각행렬로 바꾸어 주어야 한다. 우선 특이 값을 s에 저장하고 대각 행렬 크기의 행렬을 생성한 후에 그 행렬에 특이값을 삽입하자.  

In [35]:
S =np.zeros((4,9))
S[:4,:4] = np.diag(s) # 오 이런 방법이?..
print(S.round(2))
np.shape(S)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


(4, 9)

4x9 크기를 가지는 대각행렬 S가 생성되었다.  2.69 > 2.05 > 1.73 > 0.77 순으로 값이 내림차순을 보이는 것을 확인할 수 있다.

In [36]:
print(VT.round(2))
np.shape(VT)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]


(9, 9)

9 × 9의 크기를 가지는 직교 행렬 VT(V의 전치 행렬)가 생성되었습니다. 즉, U × S × VT를 하면 기존의 행렬 A가 나와야 합니다.  
Numpy의 allclose()는 2개의 행렬이 동일하면 True를 리턴합니다. 이를 사용하여 정말로 기존의 행렬 A와 동일한지 확인해보겠습니다.

In [38]:
np.allclose(A, np.dot(np.dot(U,S), VT).round(2))

True

In [40]:
np.dot(np.dot(U,S), VT).round(2)

array([[ 0.,  0.,  0.,  1., -0.,  1.,  1.,  0.,  0.],
       [ 0., -0., -0.,  1.,  1., -0.,  1., -0., -0.],
       [ 0.,  1.,  1., -0.,  2., -0., -0.,  0.,  0.],
       [ 1., -0.,  0.,  0., -0.,  0., -0.,  1.,  1.]])

지금까지 수행한 것은 풀 SVD(Full SVD)입니다. 이제 t를 정하고, 절단된 SVD(Truncated SVD)를 수행해보도록 합시다. 여기서는 t=2로 하겠습니다. 우선 대각 행렬 S 내의 특이값 중에서 상위 2개만 남기고 제거해보도록 하겠습니다.

In [41]:
S=S[:2,:2]
print(S.round(2))

[[2.69 0.  ]
 [0.   2.05]]


상위 2개의 값만 남기고 나머지는 모두 제거된 것을 볼 수 있습니다. 이제 직교 행렬 U에 대해서도 2개의 열만 남기고 제거합니다.

In [42]:
U=U[:,:2]
print(U.round(2))

[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


2개의 열만 남기고 모두 제거가 된 것을 볼 수 있습니다. 이제 행렬 V의 전치 행렬인 VT에 대해서 2개의 행만 남기고 제거합니다. 이는 V관점에서는 2개의 열만 남기고 제거한 것이 됩니다.

In [43]:
VT=VT[:2,:]
print(VT.round(2))

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


이제 축소된 행렬 U, S, VT에 대해서 다시 U × S × VT연산을 하면 기존의 A와는 다른 결과가 나오게 됩니다. 값이 손실되었기 때문에 이 세 개의 행렬로는 이제 기존의 A행렬을 복구할 수 없습니다. U × S × VT연산을 해서 나오는 값을 A_prime이라 하고 기존의 행렬 A와 값을 비교해보도록 하겠습니다.

In [44]:
A_prime=np.dot(np.dot(U,S), VT)
print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


대체적으로 기존에 0인 값들은 0에 가가운 값이 나오고, 1인 값들은 1에 가까운 값이 나오는 것을 볼 수 있습니다. 또한 값이 제대로 복구되지 않은 구간도 존재해보입니다. 

## 핵심!!!!
이제 이렇게 차원이 축소된 U, S, VT의 크기가 어떤 의미를 가지고 있는지 알아봅시다.

축소된 U는 4 × 2의 크기를 가지는데, 이는 잘 생각해보면 문서의 개수 × 토픽의 수 t의 크기입니다. 단어의 개수인 9는 유지되지 않는데 문서의 개수인 4의 크기가 유지되었으니 4개의 문서 각각을 2개의 값으로 표현하고 있습니다. 즉, U의 각 행은 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터라고 볼 수 있습니다. 축소된 VT는 2 × 9의 크기를 가지는데, 이는 잘 생각해보면 토픽의 수 t × 단어의 개수의 크기입니다. VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터라고 볼 수 있습니다.

이 문서 벡터들과 단어 벡터들을 통해 다른 문서의 유사도, 다른 단어의 유사도, 단어(쿼리)로부터 문서의 유사도를 구하는 것들이 가능해집니다.

### 실습

위처럼 쪼개는게 어떤 효과가 있을지 감이 안잡힌다.. 실습에서 알아보자

In [45]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [46]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [47]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [48]:
# 전처리 
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [54]:
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화

In [55]:
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [56]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

In [58]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)

20

In [59]:
np.shape(svd_model.components_)

(20, 1000)

In [60]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(svd_model.components_,terms)

Topic 1: [('just', 0.20887), ('like', 0.20469), ('know', 0.19349), ('people', 0.18318), ('think', 0.1697)]
Topic 2: [('thanks', 0.32763), ('windows', 0.28786), ('card', 0.18019), ('drive', 0.16864), ('mail', 0.15261)]
Topic 3: [('game', 0.34011), ('team', 0.30311), ('year', 0.26894), ('games', 0.23784), ('drive', 0.17472)]
Topic 4: [('drive', 0.46159), ('scsi', 0.17188), ('disk', 0.14451), ('hard', 0.13805), ('problem', 0.12763)]
Topic 5: [('drive', 0.39993), ('know', 0.28768), ('thanks', 0.24917), ('does', 0.24678), ('just', 0.17387)]
Topic 6: [('just', 0.55559), ('like', 0.23559), ('windows', 0.23078), ('know', 0.15795), ('does', 0.11156)]
Topic 7: [('just', 0.43264), ('like', 0.22858), ('mail', 0.15052), ('bike', 0.11698), ('thanks', 0.10025)]
Topic 8: [('does', 0.39692), ('know', 0.25192), ('chip', 0.22492), ('like', 0.17824), ('card', 0.15695)]
Topic 9: [('like', 0.42065), ('card', 0.32249), ('sale', 0.20267), ('video', 0.1571), ('offer', 0.14119)]
Topic 10: [('like', 0.61166), ('

### 5. LSA 의 장단점 (Pros and Cons of LSA)

정리해보면 LSA는 쉽고 빠르게 구현이 가능할 뿐만 아니라 단어의 잠재적인 의미를 이끌어낼 수 있어 문서의 유사도 계산 등에서 좋은 성능을 보여준다는 장점을 갖고 있다.  
하지만 SVD 특성상 이미 계산된 LSA에 새로운 데이터를 추가하여 계산하려고 하면 보통 처움부터 다시 계산해야된다. 즉, 새로운 정보에 대해 업데이트가 어렵다.  
그렇기에 Word2Vec한테 밀림.